In [148]:
import os
import requests
import json
import numpy as np
import tensorflow as tf
import pandas as pd
from config import api_key
from pprint import pprint
from sklearn.model_selection import train_test_split
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.layers import Dense
from tensorflow.keras.datasets import mnist
from sklearn.metrics import classification_report

In [149]:
url = "https://play.limitlesstcg.com/api"
api_key = "key="+ api_key
query = "game=PTCG&format=standard&"
tournamentid = ""

In [150]:
response = requests.get(url+"/tournaments/"+tournamentid+"?"+query+api_key)

In [151]:
tournaments = response.json()
tournaments

[{'game': 'PTCG',
  'name': 'Late Night #103 on PTCG Live | Road to Japan',
  'date': '2023-05-30T16:00:00.000Z',
  'format': 'STANDARD',
  'id': '6406779924ea2b561a98792c',
  'players': 177},
 {'game': 'PTCG',
  'name': 'Free entry ║ Deck Out Mondays ║ 200 codes',
  'date': '2023-05-30T00:00:00.000Z',
  'format': 'STANDARD',
  'id': '646bf66c9b9f5808497a8464',
  'players': 92},
 {'game': 'PTCG',
  'name': "Sunny's Weekly #105",
  'date': '2023-05-29T22:05:00.000Z',
  'format': 'STANDARD',
  'id': '646c22f09b9f5808497a8618',
  'players': 60},
 {'game': 'PTCG',
  'name': 'FalaMeowth! "5/0 Hero" #11',
  'date': '2023-05-29T22:00:00.000Z',
  'format': 'STANDARD',
  'id': '64746f199b9f5808497aace0',
  'players': 22},
 {'game': 'PTCG',
  'name': 'Standard Pumpka Weekly',
  'date': '2023-05-29T18:00:00.000Z',
  'format': 'STANDARD',
  'id': '6473dad59b9f5808497aab90',
  'players': 64},
 {'game': 'PTCG',
  'name': 'Torédo Cup May #4',
  'date': '2023-05-29T15:30:00.000Z',
  'format': 'STANDAR

In [152]:
cards= []
cards.append("archetype")
carddf = pd.DataFrame([], columns=[cards])
carddf["archetype"].astype(str)
carddf.head()

,archetype


In [153]:
tournament_names = []
decklists = []
deckarch = []
card_archetypes = []

for tournament in tournaments:
    tournament_names.append(tournament["name"])
    tournament_ID = tournament["id"]
    print(url+"/tournaments/"+tournament_ID+"/standings?"+query+api_key)
    response = requests.get(url+"/tournaments/"+tournament_ID+"/standings?"+query+api_key)
    data=response.json()
    data
    print(tournament["name"])
    for player in data:
        if player["decklist"] is not None:
            if player["deck"]["id"] != "other":
                idx = carddf.index.tolist() + ['end']
                carddf = carddf.reindex(idx, fill_value=0).reset_index(drop=True)
                if "pokemon" in player["decklist"]:
                    for card in player["decklist"]["pokemon"]:
                        poke_cardnumber=card["set"]+card["number"]
                        carddf.loc[len(carddf.index)-1, poke_cardnumber] = card["count"]
                if "trainer" in player["decklist"]:
                    for card in player["decklist"]["trainer"]:
                        poke_cardnumber=card["set"]+card["number"]
                        carddf.loc[len(carddf.index)-1, poke_cardnumber] = card["count"]      
                
                if "energy" in player["decklist"]:
                    for card in player["decklist"]["energy"]:
                        if card["number"].isnumeric():
                            poke_cardnumber=card["set"]+card["number"]
                            carddf.loc[len(carddf.index)-1, poke_cardnumber] = card["count"]    
                        else:  
                            poke_cardnumber="energy"+card["number"]
                            carddf.loc[len(carddf.index)-1, poke_cardnumber] = card["count"]
                carddf.loc[len(carddf.index)-1, "archetype"] = player["deck"]["id"]
                if player["deck"]["id"] not in card_archetypes:
                    card_archetypes.append(player["deck"]["id"])
carddf=carddf.fillna(0)


https://play.limitlesstcg.com/api/tournaments/6406779924ea2b561a98792c/standings?game=PTCG&format=standard&key=5ff01485ba33aa465c6d4bdc23d94e81
Late Night #103 on PTCG Live | Road to Japan
https://play.limitlesstcg.com/api/tournaments/646bf66c9b9f5808497a8464/standings?game=PTCG&format=standard&key=5ff01485ba33aa465c6d4bdc23d94e81
Free entry ║ Deck Out Mondays ║ 200 codes
https://play.limitlesstcg.com/api/tournaments/646c22f09b9f5808497a8618/standings?game=PTCG&format=standard&key=5ff01485ba33aa465c6d4bdc23d94e81
Sunny's Weekly #105
https://play.limitlesstcg.com/api/tournaments/64746f199b9f5808497aace0/standings?game=PTCG&format=standard&key=5ff01485ba33aa465c6d4bdc23d94e81
FalaMeowth! "5/0 Hero" #11
https://play.limitlesstcg.com/api/tournaments/6473dad59b9f5808497aab90/standings?game=PTCG&format=standard&key=5ff01485ba33aa465c6d4bdc23d94e81
Standard Pumpka Weekly
https://play.limitlesstcg.com/api/tournaments/6457b0ae9b9f5808497a2923/standings?game=PTCG&format=standard&key=5ff01485ba33

In [154]:
display(carddf.head())

display(carddf.tail())

,archetype,ASR60,SIT67,SIT68,SVI86,CRE61,CEL16,LOR74,CEL11,ASR46,...,LOR36,BST33,LOR37,CRZ29,LOR34,LOR40,SVI16,EVS66,BST9,SIT54
0,gardevoir-ex-sv,3.0,1.0,4.0,2.0,2.0,2.0,1.0,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,lugia-archeops,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,giratina-lz-box,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,gardevoir-ex-sv,3.0,1.0,4.0,2.0,2.0,2.0,1.0,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,gardevoir-ex-sv,3.0,1.0,4.0,2.0,2.0,2.0,1.0,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


,archetype,ASR60,SIT67,SIT68,SVI86,CRE61,CEL16,LOR74,CEL11,ASR46,...,LOR36,BST33,LOR37,CRZ29,LOR34,LOR40,SVI16,EVS66,BST9,SIT54
2145,inteleon-urshifu,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2146,goodra-lz-box,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2147,gardevoir-ex-sv,0.0,4.0,4.0,2.0,1.0,1.0,0.0,2.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2148,hisuian-zoroark-vstar,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2149,lost-zone-box,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [155]:
# print(carddf)
print(carddf.sum(axis=1, numeric_only=True).sum())

print(carddf["archetype"])


129000.0
                  archetype
0           gardevoir-ex-sv
1            lugia-archeops
2           giratina-lz-box
3           gardevoir-ex-sv
4           gardevoir-ex-sv
...                     ...
2145       inteleon-urshifu
2146          goodra-lz-box
2147        gardevoir-ex-sv
2148  hisuian-zoroark-vstar
2149          lost-zone-box

[2150 rows x 1 columns]


In [156]:
target = carddf["archetype"]
target_names = card_archetypes

In [157]:
# Get the features. 
data = carddf.drop("archetype", axis=1)
feature_names = data.columns
data.head()

,ASR60,SIT67,SIT68,SVI86,CRE61,CEL16,LOR74,CEL11,ASR46,BRS41,...,LOR36,BST33,LOR37,CRZ29,LOR34,LOR40,SVI16,EVS66,BST9,SIT54
0,3.0,1.0,4.0,2.0,2.0,2.0,1.0,1.0,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,3.0,1.0,4.0,2.0,2.0,2.0,1.0,1.0,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,3.0,1.0,4.0,2.0,2.0,2.0,1.0,1.0,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [158]:
X_train, X_test, y_train, y_test = train_test_split(data, target, random_state=42)

In [159]:
# Support vector machine linear classifier
from sklearn.svm import SVC 
model = SVC(kernel='linear')
model.fit(X_train, y_train.values.ravel())

SVC(kernel='linear')

In [160]:
# Model Accuracy
print('Test Acc: %.3f' % model.score(X_test, y_test))
# Calculate the classification report
from sklearn.metrics import classification_report
predictions = model.predict(X_test)
print(classification_report(y_test, predictions,
                            labels=target_names))

Test Acc: 0.987
                           precision    recall  f1-score   support

          gardevoir-ex-sv       0.98      1.00      0.99        51
           lugia-archeops       1.00      1.00      1.00        50
          giratina-lz-box       1.00      1.00      1.00        28
             mew-genesect       1.00      1.00      1.00        29
          arceus-giratina       1.00      1.00      1.00        37
            lost-zone-box       0.98      1.00      0.99       102
             mew-meloetta       1.00      1.00      1.00        27
         gardevoir-mewtwo       1.00      1.00      1.00         9
         inteleon-urshifu       1.00      1.00      1.00        15
    hisuian-zoroark-vstar       1.00      1.00      1.00         6
            goodra-lz-box       1.00      1.00      1.00        18
         arceus-duraludon       1.00      1.00      1.00        27
               spidops-ex       0.00      0.00      0.00         0
            lzb-charizard       1.00      1.0

c:\Users\d_hin\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\d_hin\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\d_hin\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\d_hin\anaconda3\lib\site-packages\sklearn\metrics\_cla

{'game': 'PTCG', 'name': 'Just Chilling Games #7 Free', 'date': '2023-05-13T18:05:00.000Z', 'format': 'STANDARD', 'id': '645ee7439b9f5808497a4938', 'players': 9}, {'game': 'PTCG', 'name': 'Card Cavern Trading Cards 2023 🏆 Standard #3', 'date': '2023-05-13T16:00:00.000Z', 'format': 'STANDARD', 'id': '643c57300ed4a405d8bd63d1', 'players': 78}, {'game': 'PTCG', 'name': 'The Clara & Jelena Charity Championship', 'date': '2023-05-13T14:15:00.000Z', 'format': 'STANDARD', 'id': '645efd669b9f5808497a4a18', 'players': 17}, {'game': 'PTCG', 'name': "Rossi's Single prize tournament #1", 'date': '2023-05-13T03:00:00.000Z', 'format': 'STANDARD', 'id': '645ebb2f9b9f5808497a4760', 'players': 3}, {'game': 'PTCG', 'name': 'TOURNAMENT OF DOOM! ZELDA! ZELDA! ZELDA!', 'date': '2023-05-13T02:05:00.000Z', 'format': 'STANDARD', 'id': '64484e520ed4a405d8bda0c2', 'players': 68}, {'game': 'PTCG', 'name': 'TanoCup #29', 'date': '2023-05-13T01:12:00.000Z', 'format': 'STANDARD', 'id': '645ee36a9b9f5808497a48e3', 'players': 17}, {'game': 'PTCG', 'name': 'Hoarders Casual #34', 'date': '2023-05-13T00:00:00.000Z', 'format': 'STANDARD', 'id': '6459920e9b9f5808497a30c0', 'players': 43}, {'game': 'PTCG', 'name': 'E&Tc Pokémon LIVE tournament', 'date': '2023-05-12T23:15:00.000Z', 'format': 'STANDARD', 'id': '645ac40a9b9f5808497a3712', 'players': 33}, {'game': 'PTCG', 'name': 'Just Chilling Games #6 Free', 'date': '2023-05-12T23:00:00.000Z', 'format': 'STANDARD', 'id': '645e7b019b9f5808497a4667', 'players': 10}, {'game': 'PTCG', 'name': 'Single Prize Scuffle!', 'date': '2023-05-12T19:00:00.000Z', 'format': 'STANDARD', 'id': '645cca5b9b9f5808497a4025', 'players': 24}

In [161]:
test_tournament_id = "645ac40a9b9f5808497a3712"
testresponse = requests.get(url+"/tournaments/"+test_tournament_id+"/standings?"+api_key)
print(testresponse.json())
testdata=testresponse.json()
# testcards.append("archetype")
testcarddf = pd.DataFrame([], columns=[testcards])
# testcarddf["archetype"].astype(str)
testcarddf.head()
test_tournament_names = []
decklists = []
deckarch = []


testdata
for player in testdata:
    if player["decklist"] is not None:
        if player["deck"]["id"] != "other":
            idx = testcarddf.index.tolist() + ['end']
            testcarddf = testcarddf.reindex(idx, fill_value=0).reset_index(drop=True)
            if "pokemon" in player["decklist"]:
                for card in player["decklist"]["pokemon"]:
                    poke_cardnumber=card["set"]+card["number"]
                    testcarddf.loc[len(testcarddf.index)-1, poke_cardnumber] = card["count"]
            if "trainer" in player["decklist"]:
                for card in player["decklist"]["trainer"]:
                    poke_cardnumber=card["set"]+card["number"]
                    testcarddf.loc[len(testcarddf.index)-1, poke_cardnumber] = card["count"]      
            
            if "energy" in player["decklist"]:
                for card in player["decklist"]["energy"]:
                    if card["number"].isnumeric():
                        poke_cardnumber=card["set"]+card["number"]
                        testcarddf.loc[len(testcarddf.index)-1, poke_cardnumber] = card["count"]    
                    else:  
                        poke_cardnumber="energy"+card["number"]
                        testcarddf.loc[len(testcarddf.index)-1, poke_cardnumber] = card["count"]

testcarddf=testcarddf.fillna(0)




[{'name': 'nik993', 'country': 'IT', 'decklist': {'pokemon': [{'count': 1, 'set': 'SSP', 'number': '156', 'name': 'Greninja V-UNION'}, {'count': 1, 'set': 'SSP', 'number': '157', 'name': 'Greninja V-UNION'}, {'count': 1, 'set': 'SSP', 'number': '155', 'name': 'Greninja V-UNION'}, {'count': 1, 'set': 'SSP', 'number': '158', 'name': 'Greninja V-UNION'}, {'count': 3, 'set': 'ASR', 'number': '39', 'name': 'Origin Forme Palkia V'}, {'count': 3, 'set': 'ASR', 'number': '40', 'name': 'Origin Forme Palkia VSTAR'}, {'count': 2, 'set': 'BRS', 'number': '40', 'name': 'Lumineon V'}, {'count': 1, 'set': 'SHF', 'number': '24', 'name': 'Manaphy'}, {'count': 1, 'set': 'EVS', 'number': '31', 'name': 'Suicune V'}, {'count': 1, 'set': 'ASR', 'number': '46', 'name': 'Radiant Greninja'}], 'trainer': [{'count': 3, 'set': 'CRE', 'number': '146', 'name': 'Melony'}, {'count': 2, 'set': 'BRS', 'number': '132', 'name': "Boss's Orders"}, {'count': 2, 'set': 'ASR', 'number': '147', 'name': 'Irida'}, {'count': 2, '

In [162]:
print(testcarddf.sum(axis=1, numeric_only=True).sum())

predictions = model.predict(X_test)

1860.0
